In [ ]:
from statistics import mean
import pandas as pd
from datasets import Dataset
from ragas.metrics import faithfulness
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from langchain_mistralai import ChatMistralAI

# 1. Load your Excel file
df = pd.read_excel("Questions_with_answers.xlsx")

# 2. Prepare dataset for RAGAS
dataset = Dataset.from_dict({
    "question": df["Questions"].tolist(),
    "answer": df["Answer"].tolist(),
    # RAGAS expects a list of contexts for each question
    "contexts": df["Retrieved Docs"].apply(lambda x: [str(x)]).tolist()
})

# 3. Wrap Mistral as a LangChain LLM for RAGAS
mistral_llm = ChatMistralAI(
    api_key="#####",
    model="mistral-medium-latest",
    temperature=0
)
llm_wrapper = LangchainLLMWrapper(mistral_llm)

# 4. Evaluate only faithfulness
results = evaluate(dataset, metrics=[faithfulness], llm=llm_wrapper)
print(results)
print(mean(results["faithfulness"]))
# 5. Save results back to Excel
df["Faithfulness"] = results["faithfulness"]
df.to_excel("Questions_with_faithfulness.xlsx", index=False)

print("Faithfulness scores saved to Questions_with_faithfulness.xlsx")
